In [2]:
import pydoc
import os
import fnmatch
import inspect
import sys
import importlib
from IPython.display import display, Markdown, HTML

def find_py_files(proj_root):
    py_files = []
    sub_dirs = [
        'giga/models',
        'giga/app',
    ]
    for sub_dir in sub_dirs:
        for root, _, filenames in os.walk(os.path.join(proj_root, sub_dir)):
            for filename in fnmatch.filter(filenames, '*.py'):
                py_files.append(os.path.join(root, filename))
    return py_files

project_path = os.path.join(os.getcwd(), "../..")  # Adjust the path accordingly to point to your project root folder
sys.path.append(project_path)
python_files = find_py_files(project_path)

for file_path in python_files:
    html = '<style> tr { background:none !important; } * { text-align: left !important; }</style>'
    html += f"<details><summary><span style=\"font-size: 20px;\">{file_path.replace(project_path, '')}</span></summary>"
    try:
        # Add the directory containing the Python file to sys.path
        file_dir = os.path.dirname(file_path)
        if file_dir not in sys.path:
            sys.path.append(file_dir)

        module_name = os.path.splitext(os.path.basename(file_path))[0]
        spec = importlib.util.spec_from_file_location(module_name, file_path)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)

        # Iterate and add to HTML
        for name, obj in module.__dict__.items():
            if name.startswith('_'):
                continue
            if not inspect.isroutine(obj) and not inspect.isclass(obj):
                continue
            try:
                html += f"<div>{pydoc.html.document(obj)}</div>"
            except Exception as e:
                print(f"Error generating documentation for {name}: {e}")
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    html += '</details>'
    display(HTML(html, metadata=dict(isolated=True)))
